# SKT-koBERT

https://github.com/SKTBrain/KoBERT

## Install & Library

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import os

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.5f}".format(x)})

# CPU or GPU
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
device

'cuda:1'

## Path

In [2]:
model_path = '/home/hyejeongeun/chatbot/Model/Model/'
open_path = '/home/hyejeongeun/chatbot/Data/Review/Review_mecab/'
save_path = '/home/hyejeongeun/chatbot/Data/Review/Review_embedding/'

## Model & Tokenizer

In [3]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [4]:
# 내장된 tokenizer와 sentencepiece 함수를 쓰는 듯
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        #self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i]) #+ (self.labels[i], ))

    def __len__(self):
        return (len(self.sentences))

In [6]:
## Setting parameters
max_len = 210 #210 #수정
batch_size = 16 # 64 #수정
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [7]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, #2 #수정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        
        classifier = self.classifier(pooler)
        
        if self.dr_rate:
            out = self.dropout(classifier)
  
        return pooler, out

In [8]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [9]:
model.load_state_dict(torch.load(model_path+'epoch_5_qna_sep_model_4.pt'))

<All keys matched successfully>

In [10]:
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [11]:
def review_BERTClassifier(_dataloader):
    for batch_id, (token_ids, valid_length, segment_ids) in enumerate(_dataloader):

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length  

        embedding, probability = model.forward(token_ids, valid_length, segment_ids)
        embedding = np.array(embedding.tolist())
        probability = np.array(probability.tolist())

        if batch_id == 0:
            review_embedding = embedding
            review_probability = probability
        else:
            review_embedding = np.concatenate([review_embedding, embedding], axis=0)
            review_probability = np.concatenate([review_probability, probability], axis=0)

        del embedding, probability
        torch.cuda.empty_cache() # GPU 캐시 삭제
        
    # max index 추출
    review_class = np.argmax(review_probability, axis=1)
    
    return review_embedding, review_class

## Review Classification

In [12]:
review_list = os.listdir(open_path)
review_name = [review.split('_mecab')[0] for review in review_list]

In [13]:
len(review_list)

146

In [14]:
for i in tqdm_notebook(range(50, len(review_list))): # 50부터 다시 돌림
    if review_list[i] == '.ipynb_checkpoints':
        continue
    
    # load data
    df = pd.read_csv(open_path + review_list[i])
    df=df[df['comment_mecab'].notna()]
    df.reset_index(drop=True, inplace=False)
    
    tokens = df['comment_mecab'].tolist()
    
    data = BERTDataset(tokens, 0, tok, max_len, True, False)
    _dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, num_workers=5)
    
    # classifier
    review_embedding, review_class = review_BERTClassifier(_dataloader)
    
    # length 체크
    if len(df)!=len(review_embedding.tolist()):
        print('not match length:', i)
    
    # csv 파일로 내보내기
    df['review_embedding'] = review_embedding.tolist()
    df['review_class'] = review_class.tolist()
    
    df.to_csv(save_path+review_name[i]+'_embedding'+'.csv', index=False)

/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/96 [00:00<?, ?it/s]

In [15]:
# 확인
make_list = os.listdir(save_path)

In [16]:
len(make_list)

145

## 전체 개수 확인

In [3]:
make_list = os.listdir(save_path)

In [4]:
len(make_list)

145

In [5]:
review_count = 0
for file in make_list:
    df = pd.read_csv(save_path+file)
    review_count += len(df)
    
print(review_count)

710454


## Screening

In [ ]:
df = pd.read_csv(save_path+make_list[0],sep='\t')

In [ ]:
df['review_class'].unique()

In [ ]:
for i in make_list:
    df = pd.read_csv(save_path+i,sep='\t')
    print(df['review_class'].unique())